# Exercise 3


_course: quantum cryptography for beginners
<br>date: 8 september 2020
<br>author: burton rosenberg_


## Overview


In this exercise we look at the quantum one time pad. There is an exploration of what does the result of a probabilistic algorithm mean, including the complexity class RP. The class RP touches a bit on the difference between information and knowlege. While our previous adversaries were judged on how often they were correct, even if that correctness were accident, the complexity class RP introduces a deliberateness in the algorithm's reports.

The result is that two classical bits can be used to disguise entirely a quantum bit from an eavesdropper. If the communicating parties, Alice and Bob, agree in a pair of classical random bits, or a stream of $2n$ such bits when sending $n$ qubits, then they can send qubits on a public channel with perfect secrecy. 

This extends the classical result of Shannon.


First we get our imports and IBM account loaded, plus def some helpful procedures. We will continue after this following code block.


In [1]:
import time, math, random

import qiskit
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.jobstatus import JOB_FINAL_STATES, JobStatus
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

qiskit.__qiskit_version__


args_g = []

# your api token from IBM, first time run.
# after that None is good

#api_token = 'abcdefghijklmnopqrstuvwxyz'
api_token = None 

def load_or_save_IBMQ_account(api_token=None):
    global args_g
    if api_token:
        # only needs to be done once
        # then is stored in e.g. ~/.qistkit/qiskitrc
        IBMQ.save_account(api_token)
    provider = IBMQ.load_account()
    return provider

def list_backends(provider):
    global args_g
    backends = provider.backends()
    print('backends available:')
    for be in backends:
        st = be.status()
        if st.operational:
            print(f'\t{be.name()}, pending jobs:{st.pending_jobs}')

            
def run_quantum_circuit_on_backend(quantum_circuit,provider,backend, shots=16, memory=True):
    backend = provider.get_backend(backend)
    qobj = assemble(transpile(quantum_circuit, backend=backend), backend=backend, shots=shots, memory=memory)
    job = backend.run(qobj)
    return job


def wait_for_job(backend, job, wait_interval=5):
    backend = provider.get_backend(backend)
    retrieved_job = backend.retrieve_job(job.job_id())
    start_time = time.time()
    job_status = job.status()
    while job_status not in JOB_FINAL_STATES:
        print(f'Status @ {time.time() - start_time:0.0f} s: {job_status.name},'
              f' est. queue position: {job.queue_position()}')
        time.sleep(wait_interval)
        job_status = job.status()

print(f'Please wait will we load the account and list backends ...\n')

provider = load_or_save_IBMQ_account(api_token)
list_backends(provider)

# choose your backend

backend = 'ibmq_qasm_simulator'
#backend = 'ibmq_armonk'
#backend = 'ibmq_vigo'
#backend = 'ibmq_london'

# and so forth ... chose from the results given by provider.backends()

print(f'\n\nBackend chosen: {backend}\n\n')

Please wait will we load the account and list backends ...



/Users/ojo/anaconda3/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


backends available:
	ibmq_qasm_simulator, pending jobs:1
	ibmqx2, pending jobs:7
	ibmq_16_melbourne, pending jobs:12
	ibmq_vigo, pending jobs:12
	ibmq_ourense, pending jobs:7
	ibmq_valencia, pending jobs:11
	ibmq_armonk, pending jobs:20
	ibmq_santiago, pending jobs:26


Backend chosen: ibmq_qasm_simulator




# Quantum one time pad


In the classical one-time pad, we wished to transfer a classical bit from Alice to Bob over a channel eavesdropped by Eve, such that Eve learns nothing more about the value of the bit, except for her a priori statistical knowledge of how likely is each value of the bit.

It was shown that this is possible at the expense of one random bit shared between Alice and Bob for every bit transfered. This "pad bit" is selected uniformly, indepdently at random, and is unknown to Eve.

We now want Alice to send Bob a qubit such that Eve can learn nothing of the state of the qubit.

The full problem is that Alice can put the qubit in any state. However, we will play a reduced game. 

Alice wishes to send Bob a qubit $q$ chosen from $\{\,|0\rangle,|+\rangle\,\}$. We assume she makes her choice as a uniform random variable, that is, with probability $1/2$ Alice sends $|0\rangle$ and probability $1/2$ Alice sends $|+\rangle$. Bob receives the qubit and does whatever computation he wishes.

Eve is trying to learn which qubit Alice sent. 


## Measurements

We have measured our qubits in our circuits. The result of a measurement is a classical bit, 0 or 1.

Our measurements have been in the _computational basis_, also called the $Z$ basis. This means that if the qubit is $|0\rangle$ then the measurement result is 0, and if the qubit is $|1\rangle$ then the measurement result is 1. Very convenient, thanks mostly to the notation. However, if the qubit is in any other state, the measurement is probabilistic, and it adjusts the qubit to agree with how the measurement turned out. 

A $|+\rangle$ state qubit half the time measures  $|0\rangle$ and is then in the state $|0\rangle$ and the other half the time measures  $|1\rangle$ and is then in the state $|1\rangle$.

It is possible to measure the qubit in the $X$ basis. This basis is aligns a 0 outcome along the state $|+\rangle$. It is important to note that the 1 outcome will them be the state orthogonal to this, and be $|-\rangle$. This was also true in the  $Z$ basis. The states $|0\rangle$ and $|1\rangle$ are orthogonal, so once we are setup to measure $|0\rangle$ as 0 with certainty, then we would measure $|1\rangle$ as 1 with certainty.

To measure on the $X$ basis, do a transformation before the measurement to bring the desired measurement basis to the computational basis. This transformation is $H$. 


### Exercise A

Let Alice's qubit generation function be,
$$
G(b) = \begin{cases}
|0\rangle &\mbox{if } b = 0 \\
|+\rangle &\mbox{if } b = 1
\end{cases}
$$
This is a random function driven by the random choice $b\in\{\,0,1\,\}$. We will assume that both outcomes are equally likely.

Suppose Alice does not encrypt, so the qubit on the channel is the qubit generated, $\tilde{q} = G(b)$. Eve's first algorithm $E_0$ is to measure $\tilde{q}$ in the computational basis and then outputs a bit. 

Your exercise is to consider how $E_0$ arrives at its output to maximize the probability it guesses correctly, and to give that probability. The probability the algorithm is correct is,
$$
Pr\,\{ E_0(G(b))==b \,\}.
$$
where the probability is over the choice of $b$ and any random choices $E_0$ might take.


### Exercise B

Repeat the above except this time, Eve will measure the qubit in the $X$ basis. Call that algorithm $E_+$, and see if there is any advantage over $E_0$.
s

### Exercise C

An RP type algorithm. Write a third algorithm $E_{RP}$ gives an output of the _RP_ kind. An RP algorithm is an inherently probabilistic algorihthm that gives an error-free decision for one side of the question. It is, however only one side, in that the other possible answer is that the algorithm does not know. The opposite of Yes is not No, for these algorithms. The opposite of Yes is it might be, or it might not be. 

The probabilistic constrant for an _RP_ algorithm is that is has a maximum probability of claiming ignorance of something less than 1, and it will never answer falsely the opposite side of the question.

\begin{eqnarray*}
 Pr\,\{ E_{RP}(|0\rangle) = 1 \,\} &=& 0,\\
 Pr\,\{ E_{RP}(\tilde{q}) = 0 \,\} &\le& d,
\end{eqnarray*}

for some $d<1$, where the probability is taken over the probability of this input, the classical probability in the algorithm, and the quantum probability in the measurements.

The first probability equation states that if the alogrithm outputs a 1, then the input is surely a $|+\rangle$, since it is probability zero that it is a $|0\rangle$. The second probability equation forbids that the algorithm achieve correctness by simply never outputting a 1. With probabiliy $1-d$ it will identify the input correctly and say so. This requires that a $|+\rangle$ is presented sufficiently often. Our assumption is we expect half of the inputs to be $|+\rangle$.

Modify if neccesary your algorithms to create $E_{RP}$ with $d=3/4$.

### Exercise D

In order to increase privacy, Alice and Bob share a random bit $r\in\{\,0,1\,\}$, and before she sends the qubit $q$ she applies the transformation $X$ is $r$ is one. That is, the encrypted qubit $\tilde{q}$ of qubit $q$ is,
$$
\tilde{q} = X^{r} q.
$$
Bob can recover $q$ because he knows $r$ and,
$$
X^{r} = X^{r} \tilde{q} =  X^{r} X^{r} q = q.
$$
How does this affect the power of $E_0, E_+$ or $E_{RP}$?

Try the same using,
$$
\tilde{q} = Z^{r} q.
$$


_Hint:_ $X |+\rangle = |+\rangle$ and $Z|0\rangle=|0\rangle$.


### Exercise E

This is the idea that works. Alice and Bob share two random bits, $r$ and $s$. Alice encrypts,
$$
\tilde{q} = Z^{s} X^{r}  q
$$
and Bob decrypts,
$$
q =  X^{r} Z^{s} \tilde{q}
$$
The decryption works because,
$$
X^{r} Z^{s} \tilde{q} = X^{r} Z^{s} Z^{s}  X^{r} q = X^{r} X^{r} q  = q.
$$
What is Eve's best algorithms.

_Hint:_ Eve should have no advantage over just guessing.


### Exercise F

We have shown that two classical secret bits are sufficient to hide from Eve whether a $|0\rangle$ or $|+\rangle$ qubit is being sent over a public channel. However, the method that encrypts with $Z^s X^r$ in fact hides an arbitrary state from Eve. We explore this with a new state: $|\ominus_L\rangle$, and it's opposite $|\ominus_R\rangle$.

The create these states, we need a new gate, the $S$ gate, and combine it with the $H$ gate, 

$$
|\ominus_L\rangle = S \,H \,|0\rangle,\;|\ominus_R\rangle = S\, H\, |1\rangle
$$

to measure in the _circular basis_, use the inverse of this transformation and then measure in the compuational basis. This will bring the proper state to measure a certain 0 and its opposite a certain 1.

Alice's generator now produces $|0\rangle$ or $|\ominus_L\rangle$ randomly with equal probability. Show that the encryption of the previous exercise hides all information from Eve.

While the other exercises are file in the code type, you will see in the code where you need to write and mostly they are experiements; this exercise you will need to write more code yourself, for your experiments.

The fact is that the two bits make a one time pad that transmits and arbitrary state from Alice to Bob without Eve learning anything. While the other exercises just did the 0 and plus basis, this introduces a new basis, but what you want to get the idea of is that any basis is secure. There are an infinity of basis so we would actually need a proof, but for now we just want to experiement with actual examples.



In [2]:

class Alice:
    
    def __init__(self,encrypt):
        """
        encrypt is an integer. 
        the mapping from intergers to encryptions is given by the self.encryption function vector
        """
        assert (encrypt in range(4))
        self.encryptions = [ self.encrypt_none, self.encrypt_by_x, self.encrypt_by_z, self.encrypt_by_xz]
        self.encryption_names = ["none","X","Z","XZ"]
        self.encrypt = encrypt 
        pass
    
    def gen_0_plus(self):
        """
        random source of 0/+ qubits. returns the random bit for the purpose
        of scoring Eve's guessing strategy.
        """
        qc = QuantumCircuit(1)
        r = random.randint(0,1)
        if r==1:
            qc.h(0)
        return (r,qc)
    
    def gen_0_circ(self):
        """
        random source of 0/circular qubits. returns the random bit for the purpose
        of scoring Eve's guessing strategy.
        """
        qc = QuantumCircuit(1)
        r = random.randint(0,1)
        if r==1:
            qc.h(0)
            qc.s(0)
        return (r,qc)
    
    def encrypt_none(self,qc):
        """
        none encryption. just return the source bit
        """
        return (qc,0)

    def encrypt_by_x(self,qc):
        """
        randomly applies an X gate or not to the qubit source qc
        """
        r = random.randint(0,1)
        if r==1:
            qc.x(0)
        return (qc,r)

    def encrypt_by_z(self,qc):
        """
        randomly applies a Z gate or not to the qubit source qc
        """
        r = random.randint(0,1)
        if r==1:
            qc.z(0)
        return (qc,r)

    def encrypt_by_xz(self,qc):
        """
        randomly applies as cascade of an X gate (or none) followed by a Z gate (or none)
        to the qubit source qc
        """
        r1 = random.randint(0,1)
        r2 = random.randint(0,1)
        
        #
        #
        # code needed, return (qc,r1,r2)
        #
        #
        qc = None 
        
        return (qc,r1,r2)

    def source_qubit(self):
        (r,qc) = self.gen_0_plus()
        print(f'Alice source bits 0/+ encrypted with {self.encryption_names[self.encrypt]}')
        return (r,self.encryptions[self.encrypt](qc)[0])
        
        
class Eve:
    
    def __init__(self,provider,backend):
        self.provider = provider
        self.backend = backend
      
    def run_circuit(self,qc):
        print(f'results: waiting for results from backend {self.backend} ...')
        job = run_quantum_circuit_on_backend(qc,self.provider,self.backend)
        wait_for_job(self.backend, job)
        result = job.result()
        print(f'results: {result.get_memory()}')
        return result
        
    def strategy_null(self,alice,use_0_circ=False,trials=16):
        """
        ignore the channel and just guess randomly. this can be thought of
        as the equivalent of learning nothing from the channel, as it is not looked at
        """
        
        count = 0
        total_n = 0
        for i in range(trials):
            (r,qc) = alice.source_qubit()
            
            # ignore the circuit and just guess randomly
            qc.measure_all()  # we have to measure something to get it to run
            result = eve.run_circuit(qc)
            trace = result.get_memory()

            trial_count = 0
            for t in trace:
                total_n += 1
                if random.randint(0,1) == int(t):
                    trial_count += 1
                    count += 1
            print(f'trial {i}: success rate: {trial_count/len(trace)}')

        return count/total_n

    def strategy_zero(self,alice,use_0_circ=False,trials=16,draw_circuit=True):
        """
        measure on the 0/1 basis and guess the result of the measurement
        """
        
        count = 0
        total_n = 0
        for i in range(trials):
            (r,qc) = alice.source_qubit()
            
            qc.measure_all()
            if draw_circuit:
                self.draw_circuit(qc)
            result = eve.run_circuit(qc)
            trace = result.get_memory()
            
            trial_count = 0
            for t in trace:
                total_n += 1
                if r == int(t):
                    trial_count += 1
                    count += 1
            print(f'trial {i}: success rate: {trial_count/len(trace)}')

        return count/total_n

    def strategy_plus(self,alice,use_0_circ=False,trials=16,draw_circuit=True):
        """
        measure on the +/- basis and guess the result of the measurement
        """
        
        # student writes
        
        count = 0
        total_n = 0
        for i in range(trials):
            (r,qc) = alice.source_qubit()
            
            #
            #
            # code needed here, similar to strategy_zero but
            # measure on the plus basis (need to add a gate to
            # take + to 0 cause the qiskit measurements are always
            # in the computational basis)
            #
 
            result = eve.run_circuit(qc)
            trace = result.get_memory()
            trial_count = 0
            for t in trace:
                total_n += 1
                if r == (1-int(t)): # negate the logic
                    trial_count += 1
                    count += 1
            print(f'trial {i}: success rate: {trial_count/len(trace)}')
            
        return count/total_n

    def strategy_rp_plus(self,alice,use_0_circ=False,trials=16,draw_circuit=True):
        """
        RP language of +, when certainly a + return 1, else return 0.
        correctness is: never return 1 when qubit sent is 0, **and** what is
        the probabilty the algorithm returns a 0.
        """
        
        count = 0
        total_n = 0
        
        correctness = True
        
        #
        #
        # code needed here
        #
        #
           
        # return correctness True unless it returned a 1 on a |0> input. that means
        # it is not correct. else the count/total_n is the fraction of time it returned a
        # 0 (and will be between 0.75 and 0.5, I think )
        return (count/total_n,correctness)
    

    def draw_circuit(self,qc):
        print('\n-------- CIRCUIT ---------')
        print(qc.draw(output='text'))
        print('-------------------------\n')
 
    

In [3]:

alice = Alice(3)
eve = Eve(provider, backend)
eve.strategy_rp_plus(alice,trials=16)

Alice source bits 0/+ encrypted with XZ

-------- CIRCUIT ---------
        ┌───┐ ░ ┌─┐
   q_0: ┤ X ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
trial 0: guessing rate: 0.0, correctness: False
Alice source bits 0/+ encrypted with XZ

-------- CIRCUIT ---------
        ┌───┐ ░ ┌─┐
   q_0: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: ['0', '0', '0', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '0', '1']
trial 1: guessing rate: 0.4375, correctness: False
Alice source bits 0/+ encrypted with XZ

-------- CIRCUIT ---------
        ┌───┐ ░ ┌─┐
   q_0:

(0.5078125, False)